# LM Forest RDD


Trains a linear model forest that can be used to estimate $h_k(x), k=1,...,k$  at $X=x$ in the the conditional linear model $Y = c(x) + h_1(x)W_1 + ... + h_k(x)W_k$, where Y is a (potentially vector-valued) response and W a set of regressors.

In [27]:
install.packages("https://cran.r-project.org/src/contrib/Archive/rlang/rlang_1.0.3.tar.gz", repo=NULL, type="source")

Installing package into 'C:/Users/Roberto/AppData/Local/R/win-library/4.2'
(as 'lib' is unspecified)

Warning message in install.packages("https://cran.r-project.org/src/contrib/Archive/rlang/rlang_1.0.3.tar.gz", :
"installation of package 'C:/Users/Roberto/AppData/Local/Temp/RtmpwZRD2u/downloaded_packages/rlang_1.0.3.tar.gz' had non-zero exit status"


In [16]:


#remotes::install_github("grf-labs/grf", subdir = "r-package/grf")


Warning message in utils::untar(tarfile, ...):
"'tar.exe -xf "C:\Users\Roberto\AppData\Local\Temp\RtmpAttr1x\file3e283d0d6a80.tar.gz" -C "C:/Users/Roberto/AppData/Local/Temp/RtmpAttr1x/remotes3e285e173c0a"' returned error code 1"


sandwich (3.0-1 -> 3.0-2) [CRAN]


Installing 1 packages: sandwich

Warning message:
"package 'sandwich' is in use and will not be installed"


✔  checking for file 'C:\Users\Roberto\AppData\Local\Temp\RtmpAttr1x\remotes3e285e173c0a\grf-labs-grf-19e2b66\r-package\grf/DESCRIPTION'
─  preparing 'grf': (396ms)
✔  checking DESCRIPTION meta-information
─  cleaning src
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building 'grf_2.1.0.tar.gz'
   


Installing package into 'C:/Users/Roberto/AppData/Local/R/win-library/4.2'
(as 'lib' is unspecified)

Warning message in i.p(...):
"installation of package 'C:/Users/Roberto/AppData/Local/Temp/RtmpAttr1x/file3e2823d1c9a/grf_2.1.0.tar.gz' had non-zero exit status"


In [1]:
library(rdd)
library(grf)

Warning message:
"package 'rdd' was built under R version 4.2.1"
Loading required package: sandwich

Warning message:
"package 'sandwich' was built under R version 4.2.1"
Loading required package: lmtest

Loading required package: zoo


Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric


Loading required package: AER

Warning message:
"package 'AER' was built under R version 4.2.1"
Loading required package: car

Warning message:
"package 'car' was built under R version 4.2.1"
Loading required package: carData

Warning message:
"package 'carData' was built under R version 4.2.1"
Loading required package: survival

Loading required package: Formula



ERROR: Error in library(grf): 'grf' is not a valid installed package


In [4]:
# \donttest{
if (require("rdd", quietly = FALSE)) {
# Train a LM Forest to estimate CATEs in a regression discontinuity design.
# Simulate a simple example with a heterogeneous jump in the CEF.
n <- 2000
p <- 5
X <- matrix(rnorm(n * p), n, p)
Z <- runif(n, -4, 4)
cutoff <- 0
W <- as.numeric(Z >= cutoff)
tau <- pmax(0.5 * X[, 1], 0)
Y <- tau * W  + 1 / (1 + exp(2 * Z)) + 0.2 * rnorm(n)

# Compute the Imbens-Kalyanaraman MSE-optimal bandwidth for a local linear regression.
bandwidth <- IKbandwidth(Z, Y, cutoff)
# Compute kernel weights for a triangular kernel.
sample.weights <- kernelwts(Z, cutoff, bandwidth, "triangular")

# Alternatively, specify bandwith and triangular kernel weights without using the `rdd` package.
# bandwidth <- # user can hand-specify this.
# dist <- abs((Z - cutoff) / bandwidth)
# sample.weights <- (1 - dist) * (dist <= 1) / bandwidth

# Estimate a local linear regression with the running variable Z conditional on covariates X = x:
# Y = c(x) + tau(x) W + b(x) Z.
# Specify gradient.weights = c(1, 0) to target heterogeneity in the RDD coefficient tau(x).
# Also, fit forest on subset with non-zero weights for faster estimation.
subset <- sample.weights > 0
    
lmf <- lm_forest(X[subset, ], Y[subset], cbind(W, Z)[subset, ],
                 sample.weights = sample.weights[subset], gradient.weights = c(1, 0))
tau.hat <- predict(lmf)$predictions[, 1, ]

# Plot estimated tau(x) vs simulated ground truth.
plot(X[subset, 1], tau.hat)
points(X[subset, 1], tau[subset], col = "red", cex = 0.1)
}

#> 
#> Attaching package: ‘zoo’#> The following objects are masked from ‘package:base’:
#> 
#>     as.Date, as.Date.numeric# }



Loading required package: rdd

Warning message:
"package 'rdd' was built under R version 4.2.1"
Loading required package: sandwich

Warning message:
"package 'sandwich' was built under R version 4.2.1"
Loading required package: lmtest

Loading required package: zoo


Attaching package: 'zoo'


The following objects are masked from 'package:base':

    as.Date, as.Date.numeric


Loading required package: AER

Warning message:
"package 'AER' was built under R version 4.2.1"
Loading required package: car

Warning message:
"package 'car' was built under R version 4.2.1"
Loading required package: carData

Warning message:
"package 'carData' was built under R version 4.2.1"
Loading required package: survival

Loading required package: Formula



ERROR: Error in lm_forest(X[subset, ], Y[subset], cbind(W, Z)[subset, ], sample.weights = sample.weights[subset], : no se pudo encontrar la función "lm_forest"


In [5]:
# Train a LM Forest to estimate CATEs in a regression discontinuity design.
# Simulate a simple example with a heterogeneous jump in the CEF.
n <- 2000
p <- 5
X <- matrix(rnorm(n * p), n, p)
Z <- runif(n, -4, 4)
cutoff <- 0
W <- as.numeric(Z >= cutoff)
tau <- pmax(0.5 * X[, 1], 0)
Y <- tau * W  + 1 / (1 + exp(2 * Z)) + 0.2 * rnorm(n)

# Compute the Imbens-Kalyanaraman MSE-optimal bandwidth for a local linear regression.
bandwidth <- IKbandwidth(Z, Y, cutoff)
# Compute kernel weights for a triangular kernel.
sample.weights <- kernelwts(Z, cutoff, bandwidth, "triangular")

# Alternatively, specify bandwith and triangular kernel weights without using the `rdd` package.
# bandwidth <- # user can hand-specify this.
# dist <- abs((Z - cutoff) / bandwidth)
# sample.weights <- (1 - dist) * (dist <= 1) / bandwidth

# Estimate a local linear regression with the running variable Z conditional on covariates X = x:
# Y = c(x) + tau(x) W + b(x) Z.
# Specify gradient.weights = c(1, 0) to target heterogeneity in the RDD coefficient tau(x).
# Also, fit forest on subset with non-zero weights for faster estimation.
subset <- sample.weights > 0
lmf <- lm_forest(X[subset, ], Y[subset], cbind(W, Z)[subset, ],
                 sample.weights = sample.weights[subset], gradient.weights = c(1, 0))
tau.hat <- predict(lmf)$predictions[, 1, ]

# Plot estimated tau(x) vs simulated ground truth.
plot(X[subset, 1], tau.hat)
points(X[subset, 1], tau[subset], col = "red", cex = 0.1)

ERROR: Error in lm_forest(X[subset, ], Y[subset], cbind(W, Z)[subset, ], sample.weights = sample.weights[subset], : no se pudo encontrar la función "lm_forest"


In [2]:
#Documentation

# lm_forest(
#   X,
#   Y,
#   W,
#   Y.hat = NULL,
#   W.hat = NULL,
#   num.trees = 2000,
#   sample.weights = NULL,
#   gradient.weights = NULL,
#   clusters = NULL,
#   equalize.cluster.weights = FALSE,
#   sample.fraction = 0.5,
#   mtry = min(ceiling(sqrt(ncol(X)) + 20), ncol(X)),
#   min.node.size = 5,
#   honesty = TRUE,
#   honesty.fraction = 0.5,
#   honesty.prune.leaves = TRUE,
#   alpha = 0.05,
#   imbalance.penalty = 0,
#   stabilize.splits = FALSE,
#   ci.group.size = 2,
#   compute.oob.predictions = TRUE,
#   num.threads = NULL,
#   seed = runif(1, 0, .Machine$integer.max)
# )

 ## Aplication Causal Forest RDD to following paper titled "Do Voters Affect or Elect Policies: Evidence from the u.s. House."

link [Mixtape](https://mixtape.scunning.com/06-regression_discontinuity)

In [ ]:
library(tidyverse)
library(haven)
library(estimatr)


read_data <- function(df)
{
  full_path <- paste("https://raw.github.com/scunning1975/mixtape/master/", 
                     df, sep = "")
  df <- read_dta(full_path)
  return(df)
}


lmb_data <- read_data("lmb-data.dta")


lmb_subset <- lmb_data %>% 
  filter(lagdemvoteshare>.48 & lagdemvoteshare<.52)


lm_1 <- lm_robust(score ~ lagdemocrat, data = lmb_subset, clusters = id)
lm_2 <- lm_robust(score ~ democrat, data = lmb_subset, clusters = id)
lm_3 <- lm_robust(democrat ~ lagdemocrat, data = lmb_subset, clusters = id)


summary(lm_1)
summary(lm_2)
summary(lm_3)

In [ ]:
#aggregating the data
categories <- lmb_data$lagdemvoteshare

demmeans <- split(lmb_data$score, cut(lmb_data$lagdemvoteshare, 100)) %>% 
  lapply(mean) %>% 
  unlist()

agg_lmb_data <- data.frame(score = demmeans, lagdemvoteshare = seq(0.01,1, by = 0.01))

#plotting
lmb_data <- lmb_data %>% 
  mutate(gg_group = case_when(lagdemvoteshare > 0.5 ~ 1, TRUE ~ 0))
         
ggplot(lmb_data, aes(lagdemvoteshare, score)) +
  geom_point(aes(x = lagdemvoteshare, y = score), data = agg_lmb_data) +
  stat_smooth(aes(lagdemvoteshare, score, group = gg_group), method = "lm", 
              formula = y ~ x + I(x^2)) +
  xlim(0,1) + ylim(0,100) +
  geom_vline(xintercept = 0.5)


In [ ]:
summary(lmb_data)

In [ ]:
# Compute the Imbens-Kalyanaraman MSE-optimal bandwidth for a local linear regression.
#attach(lmb_data)

cutoff <- 0.5
Y <- score
Z <- lagdemvoteshare
X <- cbind(lmb_data$realincome, lmb_data$age)
W <- as.numeric(Z >= cutoff)

bandwidth <- IKbandwidth(Z, Y, cutoff)
# Compute kernel weights for a triangular kernel.
sample.weights <- kernelwts(Z, cutoff, bandwidth, "triangular")

# Alternatively, specify bandwith and triangular kernel weights without using the `rdd` package.
# bandwidth <- # user can hand-specify this.
dist <- abs((Z - cutoff) / bandwidth)
sample.weights <- (1 - dist) * (dist <= 1) / bandwidth

# Estimate a local linear regression with the running variable Z conditional on covariates X = x:
# Y = c(x) + tau(x) W + b(x) Z.
# Specify gradient.weights = c(1, 0) to target heterogeneity in the RDD coefficient tau(x).
# Also, fit forest on subset with non-zero weights for faster estimation.
subset <- sample.weights > 0
subset <- ! is.na(subset)

In [ ]:
lmf <- lm_forest(X[subset, 1], Y[subset], cbind(W, Z)[subset, ],
                 sample.weights = sample.weights[subset], gradient.weights = c(1, 0))

plot(X[subset, 1], tau.hat)

## References

Athey, Susan, Julie Tibshirani, and Stefan Wager. "Generalized Random Forests". Annals of Statistics, 47(2), 2019.

Zeileis, Achim, Torsten Hothorn, and Kurt Hornik. "Model-based Recursive Partitioning." Journal of Computational and Graphical Statistics 17(2), 2008.